## Data Loaders

This notebook demonstrates the use of edge loader in `pyTigerGraph`. The job of a data loader is to pull data from the TigerGraph database. Currently, the following data loaders are provided:
* EdgeLoader, which returns batches of edges.
* VertexLoader, which returns batches of vertices.
* GraphLoader, which returns randomly sampled (probably disconnected) subgraphs in pandas `dataframe`, `PyG` or `DGL` format.
* NeighborLoader, which returns subgraphs using neighbor sampling in `dataframe`, `PyG` or `DGL` format.
* EdgeNeighborLoader, which returns subgraphs using neighbor sampling from edges in `dataframe`, `PyG` or `DGL` format.

Every data loader above can either get all the batches as a HTTP response (default) or stream every batch through Kafka. The former mechanism is good for testing with small graphs and it is fast, but it subjects to a data size limit of 2GB. For large graphs, the HTTP channel will likely fail due to size limit and network connectivity issues. Streaming via Kafka is offered for data robustness and scalability. Also, Kafka excels at multi-consumer use cases, and it is efficient for model search or hyperparameter tuning when there are multiiple consumers of the same data. 

The data loaders support both homogeneous and heterogenous graphs. By default, they load from all vertex and edge types and treat the graph as a homogeneous graph. But they also allow users to specify what vertex and edge types to load from and what attributes to load from each type. This way users will get heterogeneous graph outputs.

**Note**: For the data loaders to work, a UDF (User Defined Function) has to be installed into the TigerGraph database. It only needs to be installed once per database and will work for all data loaders and all graphs. 

### Connection to Database

The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. It is able to perform quite a few database tasks. Please see its [documentation](https://docs.tigergraph.com/pytigergraph/current/intro/) for details.

**Note**: Secret instead of username/password is required for TG cloud DBs created after 7/5/2022. Otherwise, you can leave it blank.

In [1]:
from pyTigerGraph import TigerGraphConnection

In [2]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

<span style="color:red">Uncomment cell below and run to get and set token if token authentication is enabled</span>. 
* This is required for all databases on tgcloud.
* `<secret>` is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.
* If you don't know your secret, you can use `secret=conn.createSecret()` to create one.

In [ ]:
#conn.getToken(<secret>)

In [3]:
# Number of vertices for every vertex type
conn.getVertexCount('*')

{'Paper': 2708}

In [4]:
# Number of edges for every type
conn.getEdgeCount()

{'Cite': 10556}

Uncomment below to install the UDF. It only needs to be installed once per database and will work for all data loaders and all graphs. **Note**: installing the UDF will overwrite any existing UDF in the database.

In [5]:
ExprFunctions="https://tg-mlworkbench.s3.us-west-1.amazonaws.com/udf/1.0/ExprFunctions.hpp"  # For enterprise users, please use the link you received.
ExprUtil=""  # For enterprise users, please use the link you received.
conn.installUDF(ExprFunctions, ExprUtil)

ExprFunctions installed succesfully


### Edge Loader

EdgeLoader pulls batches of edges from database. Specifically, it divides edges into `num_batches` and returns each batch separately. The boolean attribute provided to `filter_by` indicates which edges are included. If you need random batches, set `shuffle` to True.

**Note**: For the first time you initialize the loader on a graph in TigerGraph,
the initialization might take a minute as it installs the corresponding
query to the database and optimizes it. However, the query installation only
needs to be done once, so it will take no time when you initialize the loader
on the same TG graph again.

There are two ways to use the data loader. See
[here](https://github.com/TigerGraph-DevLabs/mlworkbench-docs/blob/main/tutorials/basics/2_dataloaders.ipynb) for examples.
* First, it can be used as an iterable, which means you can loop through it to get every batch of data. If you load all edges at once (`num_batches=1`), there will be only one batch (of all the edges) in the iterator.
* Second, you can access the `data` property of the class directly. If there is only one batch of data to load, it will give you the batch directly instead of an iterator, which might make more sense in that case. If there are multiple batches of data to load, it will return the loader again.

Args:
* attributes (list or dict, optional):
        Edge attributes to be included. If it is a list, then the attributes
        in the list from all edge types will be selected. An error will be thrown if
        certain attribute doesn't exist in all edge types. If it is a dict, keys of the 
        dict are edge types to be selected, and values are lists of attributes to be 
        selected for each edge type. Defaults to None.
* batch_size (int, optional):  
        Number of edges in each batch.  
        Defaults to None.  
* num_batches (int, optional):  
        Number of batches to split the edges.  
        Defaults to 1.  
* shuffle (bool, optional):  
        Whether to shuffle the edges before loading data.  
        Defaults to False.  
* filter_by (str, optional):
        A boolean attribute used to indicate which edges are included. Defaults to None.
* output_format (str, optional):
        Format of the output data of the loader. Only
        "dataframe" is supported. Defaults to "dataframe".
* loader_id (str, optional):
        An identifier of the loader which can be any string. It is
        also used as the Kafka topic name. If `None`, a random string will be generated
        for it. Defaults to None.
* buffer_size (int, optional):
        Number of data batches to prefetch and store in memory. Defaults to 4.
* kafka_address (str, optional):
        Address of the kafka broker. Defaults to None.
* kafka_max_msg_size (int, optional):
        Maximum size of a Kafka message in bytes.
        Defaults to 104857600.
* kafka_num_partitions (int, optional):
        Number of partitions for the topic created by this loader.
        Defaults to 1.
* kafka_replica_factor (int, optional):
        Number of replications for the topic created by this
        loader. Defaults to 1.
* kafka_retention_ms (int, optional):
        Retention time for messages in the topic created by this
        loader in milliseconds. Defaults to 60000.
* kafka_auto_del_topic (bool, optional):
        Whether to delete the Kafka topic once the
        loader finishes pulling data. Defaults to True.
* kafka_address_consumer (str, optional):
        Address of the kafka broker that a consumer
        should use. Defaults to be the same as `kafkaAddress`.
* kafka_address_producer (str, optional):
        Address of the kafka broker that a producer
        should use. Defaults to be the same as `kafkaAddress`.
* timeout (int, optional):
        Timeout value for GSQL queries, in ms. Defaults to 300000.

#### Load all edges at once directly to local. Default.

In [6]:
%%time
edge_loader = conn.gds.edgeLoader(
    num_batches=1,
    attributes=["time", "is_train"])

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 227 ms, sys: 48.6 ms, total: 276 ms
Wall time: 45.9 s


In [7]:
%%time
# Get the only batch of data via the `data` property
data = edge_loader.data
data.shape

CPU times: user 11.9 ms, sys: 3.22 ms, total: 15.1 ms
Wall time: 36.8 ms


(10556, 4)

In [8]:
data.head()

,source,target,time,is_train
0,10485760,14680110,0,0
1,10485761,1048637,0,1
2,10485761,32505881,0,1
3,10485762,12583005,0,0
4,10485763,14680101,0,1


#### Get batches of edges through http

In [9]:
%%time
edge_loader = conn.gds.edgeLoader(
    num_batches=10,
    attributes=["time", "is_train"],
    shuffle=True,
    filter_by=None
)

CPU times: user 4.46 ms, sys: 1.35 ms, total: 5.81 ms
Wall time: 8.78 ms


In [10]:
%%time
for i, batch in enumerate(edge_loader):
    print("----Batch {}: Shape {}----".format(i, batch.shape))
    print(batch.head(1))

----Batch 0: Shape (1010, 4)----
    source    target  time  is_train
0  1048576  23068756     0         1
----Batch 1: Shape (1119, 4)----
    source    target  time  is_train
0  4194309  31457366     0         1
----Batch 2: Shape (1072, 4)----
     source    target  time  is_train
0  10485763  14680101     0         1
----Batch 3: Shape (1008, 4)----
    source    target  time  is_train
0  4194310  20971534     0         0
----Batch 4: Shape (1115, 4)----
     source   target  time  is_train
0  11534338  3145787     0         0
----Batch 5: Shape (983, 4)----
    source    target  time  is_train
0  3145737  18874400     0         1
----Batch 6: Shape (1103, 4)----
    source    target  time  is_train
0  7340042  26214420     0         1
----Batch 7: Shape (1023, 4)----
    source   target  time  is_train
0  5242882  4194363     0         1
----Batch 8: Shape (1035, 4)----
    source    target  time  is_train
0  3145732  12582928     0         1
----Batch 9: Shape (1088, 4)----
     

#### For heterogeneous graphs

Since `Cora` is a homogeneous graph, we will connect to a different graph to demostrate the use case of heterogeneous graphs.

In [11]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="hetero",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

# Uncomment below to get and set token if token authentication is enabled. 
#conn.getToken(<secret>) # <secret> is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.

In [12]:
print(conn.gsql("ls"))

---- Graph hetero
Vertex Types:
- VERTEX v0(PRIMARY_ID id INT, x LIST<DOUBLE>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v1(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v2(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- DIRECTED EDGE v0v0(FROM v0, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v1(FROM v1, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v2(FROM v1, TO v2, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v0(FROM v2, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v1(FROM v2, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v2(FROM v2, TO v2, is_train BOOL, is_val BOOL)

Graphs:
- Graph hetero(v0:v, v1:v, v2:v, v0v0:e, v1v1:e, v1v2:e, v2v0:e,

In [13]:
%%time
loader = conn.gds.edgeLoader(
    attributes={"v0v0": ["is_train", "is_val"],
                "v2v0": ["is_train", "is_val"]},
    batch_size=200,
    shuffle=False,
    filter_by=None,
    loader_id=None,
    buffer_size=4
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 27.8 ms, sys: 4.36 ms, total: 32.1 ms
Wall time: 32.7 s


In [14]:
%%time
for i, batch in enumerate(loader):
    print("----Batch {}----".format(i))
    for j in batch:
        print("Edge type:", j)
        print(batch[j].head(1))

----Batch 0----
Edge type: v0v0
      source     target is_train is_val
0  137363456  136314881        0      0
Edge type: v2v0
      source     target is_train is_val
0  201326594  149946369        0      0
----Batch 1----
Edge type: v0v0
      source     target is_train is_val
0  135266305  134217729        0      0
Edge type: v2v0
      source     target is_train is_val
0  204472321  134217729        0      0
----Batch 2----
Edge type: v0v0
      source     target is_train is_val
0  135266304  147849218        0      0
Edge type: v2v0
      source     target is_train is_val
0  206569473  158334976        0      0
----Batch 3----
Edge type: v0v0
      source     target is_train is_val
0  134217728  136314884        0      0
Edge type: v2v0
      source     target is_train is_val
0  202375169  137363460        0      0
----Batch 4----
Edge type: v0v0
      source     target is_train is_val
0  157286401  154140672        0      0
Edge type: v2v0
      source     target is_train is_val


### Streaming through Kafka

**Note**: Kafka streaming is only available for the Enterprise Edition. You need to install the UDF with Kafka support.

In [ ]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

# Uncomment below to get and set token if token authentication is enabled. 
#conn.getToken(<secret>) # <secret> is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.

In [3]:
ExprFunctions="./ExprFunctions.hpp"  # For enterprise users, please use the link you received.
ExprUtil="./ExprUtil.hpp"  # For enterprise users, please use the link you received.
conn.installUDF(ExprFunctions, ExprUtil)

ExprFunctions installed succesfully
ExprUtil installed succesfully


#### Configure Kafka
Set up Kafka here. Once configured, the settings will be shared with all newly created data loaders and no need to set up Kafka for each loader. Please see official https://docs.tigergraph.com/pytigergraph/current/gds/gds for detailed settings.

In [4]:
conn.gds.configureKafka(
    kafka_address="127.0.0.1:9092",
    kafka_security_protocol="SASL_PLAINTEXT",
    kafka_sasl_mechanism="PLAIN",
    kafka_sasl_plain_username="your username",
    kafka_sasl_plain_password="your password"
)

#### Get batches of edges

In [5]:
%%time
edge_loader = conn.gds.edgeLoader(
    num_batches=10,
    attributes=["time", "is_train"],
    shuffle=True,
    filter_by=None
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 160 ms, sys: 34.8 ms, total: 195 ms
Wall time: 1min 4s


In [ ]:
%%time
for i, batch in enumerate(edge_loader):
    print("----Batch {}: Shape {}----".format(i, batch.shape))
    print(batch.head(1))